## A basic Quantum Computing Simulation
... using only NumPy, which takes ASCII-Pictures of Circuits as input.

The first (abstract) class is just a collection of basic quantum gates, together with a few helper functions.

In [2]:
import numpy as np

class QGates:
    # some numerical constants
    sq2 = np.sqrt(2.0)
    sqhalf = np.sqrt(0.5)
    
    # 1-Qubit-Gates
    One = np.array([
        [1.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j]])
    X = np.array([
        [0.+0.j, 1.+0.j],
        [1.+0.j, 0.+0.j]])
    Y = np.array([
        [0.+0.j, 0.-1.j],
        [0.+1.j, 0.+0.j]])
    Z = np.array([
        [1.+0.j, 0.+0.j],
        [0.+0.j, -1.+0.j]])
    Hadamard = sqhalf * np.array([
        [1.+0.j, 1.+0.j],
        [1.+0.j, -1.+0.j]])
    
    # 2-Qubit-Gates
    CX = np.array([
        [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j]])
    CZ = np.array([
        [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, -1.+0.j]])
    SWAP = np.array([
        [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])
    
    # 3-Qubit-Gates
    Toffoli = np.array([
        [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j]])
    
    # Each gate has an associated character. This will be important to parse the ASCII-Input into Quantum Circuits.
    def FromString(gate_string):
        if(gate_string in "+|MW"):
            return QGates.One
        elif (gate_string == "X"):
            return QGates.X
        elif (gate_string == "Y"):
            return QGates.Y
        elif (gate_string == "Z"):
            return QGates.Z
        elif (gate_string == "H"):
            return QGates.Hadamard
        
        elif (gate_string == "x"):
            return QGates.CX
        elif (gate_string == "z"):
            return QGates.CZ
        elif (gate_string == "s"):
            return QGates.SWAP
        
        elif (gate_string == "L"):
            return QGates.Toffoli
    
    # This function takes as input a square matrix small_matrix, which represents an endomorphism of a 
    # k-qubit Hilbert space, and tensors it with the identity on an n-qubit Hilbert space.
    # The result is an endomorphism of a (n+k)-qubit Hilbert space, where n+k = target_qb_length.
    # The embedding also allows for rearrangement of the qubits: the list qb_indices of length k
    # specifies which new index each of the original k qubits should get.
    def EmbedMatrix(small_matrix, target_qb_length, qb_indices):
        dim = 2 ** target_qb_length
        big_matrix = np.zeros((dim, dim), dtype=complex)
        source_qb_length = int(np.log2(len(small_matrix)))
        
        for row in range(dim):
            for col in range(dim):
                big_matrix[row,col] = small_matrix[QGates.ITf(row, qb_indices), QGates.ITf(col, qb_indices)]
                for qb in range(target_qb_length):
                    if(not qb in qb_indices) and (QGates.QBind(qb, row) != QGates.QBind(qb, col)):
                        big_matrix[row,col] = 0. + 0.j
                               
        return big_matrix
    
    # Helper function: Returns the qb_index'th digit in the binary form of the integer long_index
    def QBind(qb_index, long_index):
        return (int(long_index) & int(2 ** qb_index)) // (2**int(qb_index))
    
    # Helper function: Strings together the entries of the bit-valued list qb_indices 
    # and returns the integer obtained from interpreting this bit-string in binary representation.
    def ITf(long_idx, qb_indices):
        result = 0
        for i in range(len(qb_indices)):
            result += 2**i * QGates.QBind(qb_indices[i], long_idx)
        return result

The next class models a Layer of a Quantum Circuit. The data of such a Quantum Circuit is given by its string form. 
Among other things, the class provides methods which translate the string form into unitary matrices (the time-evolution the gates correspond to in the absence of measurements).

In [4]:
class GateLayer:
    def __init__(self, string_form):
        self.setStringForm(string_form)
    
    # Whenever the string form is changed, a few other derived variables need to be updated as well.
    def setStringForm(self, string_form):
        self.string_form = string_form.split('?')[0]
        self.is_conditional = '?' in string_form
        if(self.is_conditional):
            self.condition = int(string_form.split('?')[1])
        self.qb_count = int((len(self.string_form)+1)/2)
        self.getMatrixForm()

    # This method translates the string form into a unitary operator by combining all the appearing gates.
    # It ignores all measurement instructions.
    def getMatrixForm(self):
        self.matrix_form = np.identity(2**self.qb_count, dtype=complex)
        
        hline = False
        control_count = 0
        for i in range(len(self.string_form)):
            
            # Single-Qubit-Gates:
            if (self.string_form[i] in "XYZH"):
                self.matrix_form = np.dot(QGates.EmbedMatrix(QGates.FromString(self.string_form[i]), 
                                                          self.qb_count,
                                                          [i//2]), self.matrix_form)
            
                
            # 2-Qubit-Gates:
            if (self.string_form[i] in 'xzos'):
                hline = not hline
                # Control-Qubit:
                if (self.string_form[i] == 'o'):
                    control_index = i//2
                else: 
                    gate_letter = self.string_form[i]
                    gate_index = i//2
                if(hline):
                    control_index = i//2
                else:
                    self.matrix_form = np.dot(QGates.EmbedMatrix(QGates.FromString(gate_letter), 
                                                          self.qb_count,
                                                          [gate_index, control_index]), self.matrix_form)   
            # 3-Qubit-Gates:
            if (self.string_form[i] in ".L"):
                control_count += 1
                if (self.string_form[i] == 'L'):
                    gate_index = i//2
                if control_count == 3:
                    if (self.string_form[i] == '.'):
                        control_index = i//2
                    self.matrix_form = np.dot(QGates.EmbedMatrix(QGates.FromString("L"), 
                                                          self.qb_count,
                                                          [gate_index, control_index_2, control_index]), self.matrix_form) 
                    control_count = 0
                elif control_count == 2 and (self.string_form[i] == '.'):
                    control_index_2 = i//2
                elif control_count == 1 and (self.string_form[i] == '.'):
                    control_index = i//2
                    
        return self.matrix_form
    
    # Simulates the action of the GateLayer on the list of input states (density operators) do_start.
    # For every measurement instruction (M) in the GateLayer, the simulation doubles the list of states, 
    # such that all measurement results are taken into account.
    # On the other hand, a blind measurement instruction (W) does not result in a doubling of the list size. 
    def simulate(self, do_start):
        do_out = DOp.EvolveStack(do_start, self.matrix_form, self.condition if self.is_conditional else -1)
        for i in range(len(self.string_form[::2])):
            if (self.string_form[::2][i] == "M"):
                do_out = DOp.BranchStack(do_out,i)
            if (self.string_form[::2][i] == "W"):
                do_out = DOp.StackCollapseBlind(do_out,i)
        return do_out
    
    # Simulates the action of the GateLayer on the input state (density operator) do_start. 
    # The result of each measurement instruction encountered (M or W) is taken from the list measurement_results.
    def simulateOmniscient(self, do_start, measurement_results):
        do_out = DOp.Evolve(do_start, self.matrix_form)
        meas_nr = 0
        for i in range(len(self.string_form[::2])):
            if (self.string_form[::2][i] in "MW"):
                do_out = DOp.Collapse(do_out,i, measurement_results[meas_nr])
                meas_nr+=1
        return do_out
    
    # Simulates the action of the GateLayer on the input state (density operator) do_start. 
    # Every Measurement (M or W) is interpreted as a blind measurement.
    def simulateBlind(self, do_start):
        do_out = DOp.Evolve(do_start, self.matrix_form)
        for i in range(len(self.string_form[::2])):
            if (self.string_form[::2][i] in "MW"):
                do_out = DOp.CollapseBlind(do_out,i)
        return do_out

Multiple GateLayers assemble into a Quantum Circuit (QCirc). 
The Methods largely resemble the methods of a GateLayer.

In [ ]:
class QCirc:
    def __init__(self, layers):
        self.gate_layers=layers
    
    def FromString(string_form):
        layers = []
        for line in filter(None, string_form.split('\n')):
            layers.append(GateLayer(line))
        return QCirc(layers)
    
    # Warning: This method is only meaningful if the QCirc contains no measurement instructions, 
    # as these will be ignored.
    def getMatrix(self):
        matrix = np.identity(self.gate_layers[0].qb_count*2, dtype=complex)
        for layer in self.gate_layers:
            matrix = np.dot(layer.matrix_form, matrix)
        return matrix
    
    # Simulates the QCirc's action on the initial state (density operator) do_start.
    # Returns a list of length 2^(number of measurement instructions), whose entries are pairs
    # (probability, density operator) which contain the final density operators for chain of
    # measurement results, as well as the probability for that chain.
    def simulate(self, do_start):
        do_result = [(1., np.copy(do_start))]
        for layer in self.gate_layers:
            do_result = layer.simulate(do_result)
        return do_result
    
    # Simulates the QCirc's action on the initial state (density operator) do_start.
    # Treats all measurement instructions as blind. 
    # This method should only be used if the circuit is does not contain conditional GateLayers.
    def simulateBlind(self, do_start):
        do_result = np.copy(do_start)
        for layer in self.gate_layers:
            do_result = layer.simulateBlind(do_result)
        return do_result

The final class is again abstract. It contains methods for manipulating Density Operators, which themselves are modeled as np.arrays of square shape.

In [1]:
class DOp:
    # Returns the density operator corresponding to the n-qubit state |0...0> for n = qb_count
    def GroundState(qb_count):
        gs = np.zeros(2 ** qb_count)
        gs[0] = 1
        return DOp.FromStateVector(gs)
    
    # Returns the density operator corresponding to a state.
    def FromStateVector(statevec):
        return np.dot(np.array([statevec]).T.conjugate(), np.array([statevec]))
    
    # Attempts to reconstruct a state from a density operator (this is in general not possible).
    def StateForm(op):
        return np.array([np.real(np.sqrt(op[i,i])) for i in range(len(op[:,0]))])
    
    # Simulates the time-evolution of a density operator op described by a unitary matrix evo_matrix.
    def Evolve(op, evo_matrix):
        return np.dot(np.dot(evo_matrix, op), evo_matrix.T.conjugate())
    
    # Simulates the time-evolution of a stack (list of (probability, operator)-pairs) of density operators.
    # Only those density operators at entries in the list that match the condition are evolved.
    def EvolveStack(op_stack, evo_matrix, condition):
        result = []
        for i in range(len(op_stack)):
            if condition < 0 or QGates.QBind(int(np.log2(len(op_stack))) - condition-1, i) > 0:
                result.append((op_stack[i][0], DOp.Evolve(op_stack[i][1], evo_matrix)))
            else:
                result.append((op_stack[i][0], op_stack[i][1]))
        return result
    
    # From a stack of density operators and a list of measurement results, returns the (probability, operator)-pair for
    # this list of measurement results.
    def StackPair(op_stack, meas_results):
        idx = 0
        for i in range(len(meas_results)):
            idx += 2**i * meas_results[i]
        return op_stack[idx]
    
    # Doubles the stack op_stack, and simulates, for each entry of the stack, the collapse for each measurement result 
    # of the qubit qb_index.
    def BranchStack(op_stack, qb_index):
        return [(pair[0] * DOp.MeasureProb(pair[1], qb_index, m_res), DOp.Collapse(pair[1], qb_index, m_res)) for m_res in [0,1] for pair in op_stack]
    
    # Turns a stack into a single density operator by summation, weighted by the respective probabilities.
    # In practice, forgets all previous measurement results.
    def MeldStack(op_stack):
        op = 0. * op_stack[0][1]
        for pair in op_stack:
            op = op + pair[0] * pair[1]
        return op
    
    # Returns the matrix representing the projector onto the subspace where the qubit qb_index is in the state result.
    def Projector(qb_index, result, qb_count):
        P = np.zeros((2**qb_count, 2**qb_count))
        for i in range(2**qb_count):
            if(QGates.QBind(qb_index, i) == result):
                P[i,i] = 1
        return P
    
    # Returns the probability to measure the qubit qb_index in the state result for the density operator op.
    def MeasureProb(op, qb_index, result):
        return np.real(np.trace(np.dot(op, DOp.Projector(qb_index, result, int(np.log2(len(op)))))))
    
    # Returns the expectation value for a measurement of qb_index in the density operator op.
    def ExpVal(op, qb_index):
        return DOp.MeasureProb(op, qb_index, 1)
    
    # Applies the collapse of the density operator op associated with a measurement of the qubit qb_index in the state result.
    def Collapse(op, qb_index, result):
        P = DOp.Projector(qb_index, result, int(np.log2(len(op))))
        m_prob = DOp.MeasureProb(op, qb_index, result)
        return (1./m_prob if m_prob > 0.0000001 else 0.) * np.dot(np.dot(P, op), P)
    
    # Returns the statistical combination of a collapse of the density operator op associated with a measurement
    # of the qubit qb_index in either state.
    def CollapseBlind(op, qb_index):
        return (DOp.MeasureProb(op, qb_index, 0) * DOp.Collapse(op, qb_index, 0)) + (DOp.MeasureProb(op, qb_index, 1) * DOp.Collapse(op, qb_index, 1))
    
    # Performs CollapseBlind for each density operator in the stack op_stack.
    def StackCollapseBlind(op_stack, qb_index):
        return [(pair[0], DOp.CollapseBlind(pair[1], qb_index)) for pair in op_stack]
    